# Introduction

Pipelines are a simple way to keep your data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so you can use the whole bundle as if it were a single step.

Many data scientists hack together models without pipelines, but pipelines have some important benefits. Those include:

  - <b>Cleaner Code:</b> Accounting for data at each step of preprocessing can get messy. With a pipeline, you won't need to manually keep track of your training and validation data at each step.
  - <b>Fewer Bugs:</b> There are fewer opportunities to misapply a step or forget a preprocessing step.
  - <b>Easier to Productionize:</b> It can be surprisingly hard to transition a model from a prototype to something deployable at scale. We won't go into the many related concerns here, but pipelines can help.
  - <b>More Options for Model Validation:</b> You will see an example in the next tutorial, which covers cross-validation.


# Example

As in the previous tutorial, we will work with the Melbourne Housing dataset.

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the training and validation data in X_train, X_valid, y_train, and y_valid

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("./melb_data.csv")
y = data.Price
X = data.drop(['Price'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X,y,train_size=0.8, test_size=0.2)

In [5]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)

categorical_columns = [cname for cname in X_train_full.columns 
                       if X_train_full[cname].nunique() < 10 
                       and X_train_full[cname].dtype=='object']

numerical_columns = [cname for cname in X_train_full.columns
                     if X_train_full[cname].dtype in ['int64','float64']]

my_cols = categorical_columns+numerical_columns
X_train = X_train_full[my_cols]
X_valid = X_valid_full[my_cols]

In [6]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
6586,h,S,Southern Metropolitan,4,5.1,3181.0,4.0,1.0,1.0,230.0,NaN,NaN,-37.85490,144.99480,4380.0
882,t,S,Southern Metropolitan,3,13.9,3165.0,3.0,1.0,1.0,642.0,NaN,NaN,-37.92690,145.07870,10969.0
6257,h,S,Northern Metropolitan,2,6.5,3071.0,2.0,1.0,1.0,372.0,110.0,1950.0,-37.76310,145.00910,8870.0
616,u,S,Southern Metropolitan,3,9.2,3104.0,3.0,2.0,2.0,244.0,NaN,NaN,-37.79330,145.06720,7809.0
11796,h,SA,Northern Metropolitan,5,20.5,3752.0,5.0,3.0,4.0,700.0,NaN,NaN,-37.62166,145.06747,7969.0


We construct the full pipeline in three steps.
## Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to bundle together different preprocessing steps. The code below:

    imputes missing values in numerical data, and
    imputes missing values and applies a one-hot encoding to categorical data.



In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# preprocessing for categorical data
categorical_transformer = Pipeline(steps=
                                   [('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


preprocessor = ColumnTransformer(transformers=[('num',numerical_transformer, numerical_columns),
                                              ('cat',categorical_transformer, categorical_columns)])
                                    


# Step 2: Define the Model

Next, we define a random forest model with the familiar RandomForestRegressor class.


In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)


# Step 3: Create and Evaluate the Pipeline

Finally, we use the Pipeline class to define a pipeline that bundles the preprocessing and modeling steps. There are a few important things to notice:

   - With the pipeline, we preprocess the training data and fit the model in a single line of code. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)
    
   - With the pipeline, we supply the unprocessed features in X_valid to the predict() command, and the pipeline automatically preprocesses the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)



In [13]:
from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps = [('preprocessor',preprocessor),
                                ('model',model)])

my_pipeline.fit(X_train,y_train)

preds=my_pipeline.predict(X_valid)

score=mean_absolute_error(y_valid, preds)

print('MAE=',score)

MAE= 164956.13011957362



Conclusion

Pipelines are valuable for cleaning up machine learning code and avoiding errors, and are especially useful for workflows with sophisticated data preprocessing.